In [1]:
import sys
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

np.random.seed(0)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
DATA_FOLDER_PATH = '/content/drive/MyDrive/IFT3710/Animals_with_Attributes2/'
JPEGIMAGES_FOLDER_PATH = '/content/drive/MyDrive/IFT3710/Animals_with_Attributes2/JPEGImages/'

In [5]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['elephant', 'lion', 'deer', 'cow', 'squirrel', 'german+shepherd', 'skunk', 'horse', 'mole', 'walrus', 'weasel', 'mouse', 'buffalo', 'grizzly+bear', 'bat', 'chimpanzee', 'beaver', 'rabbit', 'wolf', 'bobcat', 'seal', 'collie', 'spider+monkey', 'otter', 'rat', 'leopard', 'zebra', 'sheep', 'blue+whale', 'ox', 'chihuahua', 'hamster', 'tiger', 'giraffe', 'polar+bear', 'dolphin', 'fox', 'siamese+cat', 'persian+cat', 'raccoon', 'antelope', 'pig', 'giant+panda', 'killer+whale', 'moose', 'dalmatian', 'humpback+whale', 'gorilla', 'rhinoceros', 'hippopotamus']


50

In [6]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH): #-> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)


{'elephant': 1038, 'lion': 1019, 'deer': 1344, 'cow': 1338, 'squirrel': 1200, 'german+shepherd': 1033, 'skunk': 188, 'horse': 1645, 'mole': 100, 'walrus': 215, 'weasel': 282, 'mouse': 185, 'buffalo': 904, 'grizzly+bear': 852, 'bat': 383, 'chimpanzee': 728, 'beaver': 193, 'rabbit': 1088, 'wolf': 589, 'bobcat': 630, 'seal': 988, 'collie': 1028, 'spider+monkey': 291, 'otter': 758, 'rat': 310, 'leopard': 720, 'zebra': 1170, 'sheep': 1420, 'blue+whale': 174, 'ox': 728, 'chihuahua': 567, 'hamster': 779, 'tiger': 877, 'giraffe': 1202, 'polar+bear': 868, 'dolphin': 946, 'fox': 664, 'siamese+cat': 500, 'persian+cat': 747, 'raccoon': 512, 'antelope': 1046, 'pig': 713, 'giant+panda': 874, 'killer+whale': 291, 'moose': 704, 'dalmatian': 549, 'humpback+whale': 709, 'gorilla': 872, 'rhinoceros': 696, 'hippopotamus': 684}
{'elephant': 0.0278, 'lion': 0.0273, 'deer': 0.036, 'cow': 0.0358, 'squirrel': 0.0321, 'german+shepherd': 0.0277, 'skunk': 0.005, 'horse': 0.0441, 'mole': 0.0027, 'walrus': 0.0058, 

In [7]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """
    
    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Sucessfully created annotations.csv file.


In [8]:
# labels_in_number = pd.read_csv(DATA_FOLDER_PATH+"classes.txt", delim_whitespace=True,header=None)
labels_dict = {}
with open(DATA_FOLDER_PATH+"classes.txt") as f:
    for line in f:
        # print(line.split())
        (key,val) = line.split()
        labels_dict[val] = int(key)-1
print(labels_dict)

{'antelope': 0, 'grizzly+bear': 1, 'killer+whale': 2, 'beaver': 3, 'dalmatian': 4, 'persian+cat': 5, 'horse': 6, 'german+shepherd': 7, 'blue+whale': 8, 'siamese+cat': 9, 'skunk': 10, 'mole': 11, 'tiger': 12, 'hippopotamus': 13, 'leopard': 14, 'moose': 15, 'spider+monkey': 16, 'humpback+whale': 17, 'elephant': 18, 'gorilla': 19, 'ox': 20, 'fox': 21, 'sheep': 22, 'seal': 23, 'chimpanzee': 24, 'hamster': 25, 'squirrel': 26, 'rhinoceros': 27, 'rabbit': 28, 'bat': 29, 'giraffe': 30, 'wolf': 31, 'chihuahua': 32, 'rat': 33, 'weasel': 34, 'otter': 35, 'buffalo': 36, 'zebra': 37, 'giant+panda': 38, 'deer': 39, 'bobcat': 40, 'pig': 41, 'lion': 42, 'mouse': 43, 'polar+bear': 44, 'collie': 45, 'walrus': 46, 'raccoon': 47, 'cow': 48, 'dolphin': 49}


In [9]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        key = self.img_labels.iloc[idx, 1]

        # Mapping the labels from string to tensor
        label = labels_dict[key]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToPILImage(),
    transforms.ToTensor()
])

dataset = AWA2Dataset()
dataset.transform=transform

In [19]:
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers =0, shuffle=False)

# GlobalSum = 0
# num_images_processed=0

# Function provided by https://www.binarystudy.com/2021/04/how-to-calculate-mean-standard-deviation-images-pytorch.html
def batch_mean_and_sd(dataloader):
    global GlobalSum
    global num_images_processed
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)

    for i, (images, labels) in enumerate(dataloader):
        b, c, h, w = images.shape
        nb_pixels = b * h * w
        # num_images_processed += b
        sum_ = torch.sum(images, dim=[0, 2, 3])
        # GlobalSum +=sum_
        sum_of_square = torch.sum(images ** 2,dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
        cnt += nb_pixels

        if i % 250 == 0 :
          print(f'processed image # {i} / {len(dataset)}')

    mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)        
    return mean,std

mean, std = batch_mean_and_sd(dataloader)
print("mean and std: \n", mean, std) 


processed image # 0 / 37340
processed image # 250 / 37340
processed image # 500 / 37340
processed image # 750 / 37340
processed image # 1000 / 37340
processed image # 1250 / 37340
processed image # 1500 / 37340
processed image # 1750 / 37340
processed image # 2000 / 37340
processed image # 2250 / 37340
processed image # 2500 / 37340
processed image # 2750 / 37340
processed image # 3000 / 37340
processed image # 3250 / 37340
processed image # 3500 / 37340
processed image # 3750 / 37340
processed image # 4000 / 37340
processed image # 4250 / 37340
processed image # 4500 / 37340
processed image # 4750 / 37340
processed image # 5000 / 37340
processed image # 5250 / 37340
processed image # 5500 / 37340
processed image # 5750 / 37340
processed image # 6000 / 37340
processed image # 6250 / 37340
processed image # 6500 / 37340
processed image # 6750 / 37340
processed image # 7000 / 37340
processed image # 7250 / 37340
processed image # 7500 / 37340
processed image # 7750 / 37340
processed imag

# Mean and Std of dataset is :    

# [0.4643, 0.4640, 0.3985] , [0.2521, 0.2425, 0.2538]